In [1]:
from datasketch import MinHash, MinHashLSH, MinHashLSHForest
import jieba.posseg as pseg
import re

In [7]:
# words segmentation
def word_seg(item_text):
    item_str = ''
    item = (pseg.cut(item_text))
    for i in list(item):
        if i.word not in list(stop):
            item_str += i.word
            item_str += ' '
    return item_str

In [12]:
# create minHash
def get_minHash(item_str):
    temp = MinHash()
    for d in item_str:
        temp.update(d.encode('utf-8'))
    return temp

In [22]:
# query top k
def query(key, top=3):
    item_str = word_seg(key)
    minhash_key = get_minHash(item_str)
    result = forest.query(minhash_key, top)
    for i in range(len(result)):
        print(result[i], minhash_key.jaccard(minhash_list[result[i]]), documents[result[i]].replace(' ', ''))
    print('Top {0} neibours {1}'.format(top, result))
    return result

In [24]:
# read file
with open('./weibos.txt', 'r', encoding='UTF-8') as f:
    text = f.read()
    sentences = re.split('[。！?]', text.replace('\n', ''))
    if sentences[len(sentences)-1] == '':
        sentences.pop()

In [30]:
# initialize stop words
stop = [line.strip() for line in open('./stopword.txt').readlines()]

In [31]:
# segement text
documents = []
for item_text in sentences:
    item_str = word_seg(item_text)
    documents.append(item_str)

In [32]:
# create LSH Forest and minhash objects
minhash_list = []
forest = MinHashLSHForest()
for i in range(len(documents)):
    temp = get_minHash(documents[i])
    minhash_list.append(temp)
    forest.add(i, temp)
forest.index()

In [33]:
key = '因为获得中国国家援助，叙利亚足球教练谴责队员赢了中国队，'
query(key, 3)

26 0.34375 ”​因为中国队给斯科拉里开足了薪水，给足了他需要的条件
21 0.2421875 中国援助了我们那么多粮食和美金，如果他们不再援助我们国家，你狗日些要吃土去
13 0.171875 当然，老头也是世界杯冠军教头，万一折在中国这里也没啥丢人的，毕竟里皮也折了嘛
Top 3 neibours [26, 21, 13]


[26, 21, 13]